# Обучение моделей

## Импорт библиотек

In [1]:
import os, sys

PROJECT_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))
print(PROJECT_DIR)
sys.path.append(PROJECT_DIR)

/workspaces/street_tree


In [2]:
import pandas as pd 
from src.config import PATH_RAW, PATH_PROCESSED, URL, NAME_DATASET, PATH_MODELS, output_path
from src.dowload_datafraime import download_file
from src.visualization import plot_corr_matrix
from src.preprocessing import (df_fillna, split_problems, convert_to_bool, encode_and_save_categorical, 
                               load_and_encode_categorical, split_and_save)


2025-03-23 08:44:25,774 - src.utils - INFO - Logger is ready
2025-03-23 08:44:25,775 - src.utils - INFO - This is a test log message.
2025-03-23 08:44:25,776 - src.utils - INFO - PROJECT_DIR: /workspaces/street_tree
2025-03-23 08:44:25,777 - src.utils - INFO - PATH_LOGS: /workspaces/street_tree/logs
2025-03-23 08:44:25,778 - src.utils - INFO - PATH_DATA: /workspaces/street_tree/data
2025-03-23 08:44:25,778 - src.utils - INFO - PATH_RAW: /workspaces/street_tree/data/raw
2025-03-23 08:44:25,779 - src.utils - INFO - PATH_PROCESSED: /workspaces/street_tree/data/processed
2025-03-23 08:44:25,779 - src.utils - INFO - PATH_MODELS: /workspaces/street_tree/models
2025-03-23 08:44:25,780 - src.utils - INFO - PATH_REPORTS: /workspaces/street_tree/reports
2025-03-23 08:44:25,781 - src.utils - INFO - PATH_DOCS: /workspaces/street_tree/docs
2025-03-23 08:44:25,782 - src.utils - INFO - PATH_SRC: /workspaces/street_tree/src
2025-03-23 08:44:25,782 - src.utils - INFO - output_path: /workspaces/street_t

## Предобработка

In [3]:
df = pd.read_csv(f'{PATH_RAW}/{NAME_DATASET}.csv')
df.head(3)

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,...,boro_ct,state,latitude,longitude,x_sp,y_sp,council district,census tract,bin,bbl
0,180683,348711,2015-08-27T00:00:00.000,3,0,OnCurb,Alive,Fair,Acer rubrum,red maple,...,4073900,New York,40.723092,-73.844215,1027431.148,202756.7687,29.0,739.0,4052307.0,4.022210e+09
1,200540,315986,2015-09-03T00:00:00.000,21,0,OnCurb,Alive,Fair,Quercus palustris,pin oak,...,4097300,New York,40.794111,-73.818679,1034455.701,228644.8374,19.0,973.0,4101931.0,4.044750e+09
2,204026,218365,2015-09-05T00:00:00.000,3,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.717581,-73.936608,1001822.831,200716.8913,34.0,449.0,3338310.0,3.028870e+09


In [4]:
df.columns = [col.lower().replace(' ', '_') for col in df.columns]
df_drop_columns = ['status', 'state', 'tree_id', 'created_at', 'stump_diam', 'address', 'spc_common',
                   'borocode', 'x_sp', 'y_sp', 'council_district', 'census_tract', 'nta_name',
                   'bin', 'zip_city', 'community_board', 'bbl']
df = df.drop(columns=df_drop_columns)
df.dropna(subset=['health'], inplace=True)
df.shape


(652172, 28)

In [5]:
df = df_fillna(df)
df.shape

/workspaces/street_tree/src/preprocessing.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['spc_latin'].fillna('No observation',inplace=True)
/workspaces/street_tree/src/preprocessing.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].metho

(652172, 28)

In [6]:
df = split_problems(df)
df.shape

2025-03-23 08:44:30,535 - src.utils - INFO - {'rootother', 'branchlights', 'stones', 'trunklights', 'wiresrope', 'branchother', 'metalgrates', 'noproblem', 'trunkother', 'sneakers'} - уникальные проблемы


(652172, 38)

In [7]:
df = convert_to_bool(df)
df.shape

2025-03-23 08:44:34,715 - src.utils - INFO - Значения преобразованы в булевые


(652172, 38)

In [8]:
drop_columns = ['trunklights', 'metalgrates', 'branchother', 'sneakers', 'wiresrope', 
    'branchlights', 'noproblem', 'rootother', 'stones', 'trunkother']
df = df.drop(columns=drop_columns)
df.shape

(652172, 28)

In [9]:
categorical_columns = df.select_dtypes(include='object').columns.tolist()
categorical_columns

['health', 'spc_latin', 'steward', 'guards', 'user_type', 'borough', 'nta']

In [10]:
df, label_encoders = encode_and_save_categorical(df, categorical_columns, PATH_MODELS)
df.shape

(652172, 28)

In [11]:
# Вывод значений, закодированных энкодером для столбца 'health', вместе с соответствующими им числовыми кодами
health_classes = label_encoders['health'].classes_
health_codes = label_encoders['health'].transform(health_classes)

# Создание DataFrame для удобного отображения
health_mapping = pd.DataFrame({'Health': health_classes, 'Code': health_codes})
print(health_mapping)

  Health  Code
0   Fair     0
1   Good     1
2   Poor     2


## Разделим на выборки

In [12]:

X_train, X_val, X_test, y_train, y_val, y_test = split_and_save(df, PATH_PROCESSED)

2025-03-23 08:44:40,856 - src.utils - INFO - Data successfully saved to: /workspaces/street_tree/data/processed


## Обучим модели

In [13]:
y_train.unique()

array([1, 2, 0])

In [14]:
from sklearn.preprocessing import StandardScaler
import joblib

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
# Save the scaler
joblib.dump(scaler, f'{PATH_MODELS}/scaler.pkl')
print("Scaler saved to scaler.pkl")

Scaler saved to scaler.pkl


In [15]:
PATH_MODELS

'/workspaces/street_tree/models'

In [16]:
%load_ext autoreload
%autoreload 2

from src.modeling import TabularNN

model = TabularNN(X_train, y_train, X_val, y_val, hidden_dims=[128, 64], model_path=f'{PATH_MODELS}/tabular_model.pth')  # Changed to a file path

model.train_model(epochs=50, learning_rate=0.0001, batch_size=64)
predictions = model.predict(X_test)
print(predictions)



2025-03-23 08:44:55,977 - src.utils - INFO - Epoch 1/50 - Train Loss: 7639.1787, Val Loss: 1590.4739, LR: 0.000100
2025-03-23 08:44:55,978 - src.utils - INFO - Accuracy: 0.6118, F1-Score: 0.6600, AUC-ROC: 0.6456
2025-03-23 08:44:55,979 - src.utils - INFO - Confusion Matrix:
3047	7352	4076
10928	55149	13251
673	1697	1653
2025-03-23 08:44:55,981 - src.utils - INFO - Best model saved based on highest AUC-ROC.
2025-03-23 08:45:08,774 - src.utils - INFO - Epoch 2/50 - Train Loss: 7482.7801, Val Loss: 1579.7830, LR: 0.000100
2025-03-23 08:45:08,774 - src.utils - INFO - Accuracy: 0.5799, F1-Score: 0.6401, AUC-ROC: 0.6544
2025-03-23 08:45:08,775 - src.utils - INFO - Confusion Matrix:
3247	6541	4687
11826	51561	15941
698	1399	1926
2025-03-23 08:45:08,777 - src.utils - INFO - Best model saved based on highest AUC-ROC.
2025-03-23 08:45:21,591 - src.utils - INFO - Epoch 3/50 - Train Loss: 7446.8696, Val Loss: 1574.2127, LR: 0.000100
2025-03-23 08:45:21,591 - src.utils - INFO - Accuracy: 0.5732, F1

[0 0 0 ... 0 0 0]


In [17]:
%load_ext autoreload
%autoreload 2
from src.modeling_catboost import CatBoostModelTrain

catboost_model = CatBoostModelTrain(model_dir=PATH_MODELS)
catboost_model.train(X_train, y_train, X_val, y_val)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


0:	learn: 1.0665979	test: 1.0666219	best: 1.0666219 (0)	total: 213ms	remaining: 3m 32s
100:	learn: 0.5507534	test: 0.5524875	best: 0.5524875 (100)	total: 16.8s	remaining: 2m 29s
200:	learn: 0.5359341	test: 0.5390441	best: 0.5390441 (200)	total: 33.5s	remaining: 2m 13s
300:	learn: 0.5285785	test: 0.5328005	best: 0.5328005 (300)	total: 50.3s	remaining: 1m 56s
400:	learn: 0.5233058	test: 0.5285680	best: 0.5285680 (400)	total: 1m 7s	remaining: 1m 40s
500:	learn: 0.5189233	test: 0.5251466	best: 0.5251466 (500)	total: 1m 23s	remaining: 1m 23s
600:	learn: 0.5150158	test: 0.5223382	best: 0.5223382 (600)	total: 1m 40s	remaining: 1m 6s
700:	learn: 0.5116505	test: 0.5199577	best: 0.5199577 (700)	total: 1m 57s	remaining: 50s
800:	learn: 0.5087287	test: 0.5180628	best: 0.5180628 (800)	total: 2m 13s	remaining: 33.2s
900:	learn: 0.5060961	test: 0.5163754	best: 0.5163754 (900)	total: 2m 30s	remaining: 16.5s


2025-03-23 08:55:46,371 - src.utils - INFO - Сохраняем модель в /workspaces/street_tree/models/catboost_model.cbm


999:	learn: 0.5037241	test: 0.5149162	best: 0.5149162 (999)	total: 2m 47s	remaining: 0us

bestTest = 0.5149161969
bestIteration = 999



2025-03-23 08:55:46,735 - src.utils - INFO - CatBoost Accuracy: 0.8163, F1-Score: 0.7480, AUC-ROC: 0.7488
2025-03-23 08:55:46,735 - src.utils - INFO - Confusion Matrix:
[[  803 13577    95]
 [  407 78871    50]
 [  309  3531   183]]
